#### EECE 5698
# Project-3
##### Devesh Tarasia (001537213)
tarasia.dev@northeastern.edu

In [2]:
import numpy as np
import pandas as pd
import math as maths
import seaborn as sns
import matplotlib.pyplot as plt
import random as rnd
from matplotlib.patches import Rectangle

In [ ]:
class Maze():
    def __init__(self,nRows,nCols,actionList,prob,startPosn,goalPosn,oilPosn,bumpPosn,wallPosn,oilR,bumpR,goalR,emptyR,actionR):
        self.nRows = nRows
        self.nCols = nCols
        self.actionList = actionList
        self.startPosn = startPosn
        self.goalPosn = goalPosn
        self.oilPosn = oilPosn
        self.bumpPosn = bumpPosn
        self.wallPosn = wallPosn
        self.oilR = oilR
        self.bumpR = bumpR
        self.goalR = goalR
        self.emptyR = emptyR
        self.actionR = actionR
        self.p = prob
        self.maze = np.zeros((self.nRows,self.nCols))
        self.values = np.zeros((self.nRows,self.nCols))
        self.policy = np.random.choice(self.actionList,(self.nRows,self.nCols))
        self.currentI = 1
        self.currentJ = 1

        self.maze[self.goalPosn] = self.goalR
        self.values[self.goalPosn] = self.goalR
        for i,j in self.oilPosn:
            self.maze[i,j] = self.oilR
        for i,j in self.bumpPosn:
            self.maze[i,j] = self.bumpR
        for i,j in self.wallPosn:
            self.maze[i,j] = 0.0

    def getActionList(self):
        return self.actionList

    def getStateValue(self,currentState): #May not be needing it
        return self.values[currentState]

    def getValue(self):
        return self.values
    
    def setValue(self,currentState,value):
        self.values[currentState] = value
    
    def getStatePolicy(self,currentState):
        return self.policy[currentState]

    def getPolicy(self):
        return self.policy

    def setStatePolicy(self,currentState,action):
        self.policy[currentState] = action

    def stateGenerator(self):
        i = 1
        j = 1
        while i < self.nRows - 1:
            if (i,j) not in self.wallPosn:
                yield (i,j)
            j = j + 1
            if j == self.nCols - 1:
                j = 1
                i = i + 1

    def reward(self,currentState,action=None):
        i,j = currentState
        if action == None:
            action = self.policy[currentState]
        probabilities = []
        futureStates = [currentState,(i,j+1),(i,j-1),(i+1,j),(i-1,j)] #current,R,L,D,U
        rewards = np.array([self.maze[k]+self.actionR for k in futureStates])
        
        if action == 'R':
            probabilities = [0,1-self.p,self.p/3,self.p/3,self.p/3]
        if action == 'L':
            probabilities = [0,self.p/3,1-self.p,self.p/3,self.p/3]
        if action == 'D':
            probabilities = [0,self.p/3,self.p/3,1-self.p,self.p/3]
        if action == 'U':
            probabilities = [0,self.p/3,self.p/3,self.p/3,1-self.p]
        
        for k in range(1,len(futureStates)): #to add probabilities if wall is in side
            if futureStates[k] in wallPosn: 
                probabilities[0] += probabilities[k]
                probabilities[k] = 0

        xaxis = [i[0] for i in futureStates]
        yaxis = [i[1] for i in futureStates]
        adjacentStates = (xaxis,yaxis)
        probs = np.array(probabilities)

        return probs, rewards, adjacentStates


gamma = 0.55
theta = 0.01
nrows = 20
ncols = 20
prob = 0.02
actionList = ['R','L','U','D']
oilRew = -5
bumpRew = -10
goalRew = 200
actionRew = -1
emptyRew = 0
goalPosn = (3,13)
startPosn = (15,4)
oilPosn = [(2,8),(2,16),(4,2),(5,6),(9,18),(15,10),(16,10),(17,14),(17,17),(18,7)]
bumpPosn = [(1,11),(1,12),(2,1),(2,2),(2,3),(5,1),(5,9),(5,17),(6,17),(7,17),(8,17),(7,10),(7,11),(7,2),(12,11),(12,12),(14,1),(14,2),(15,17),(15,18),(16,7)]
wallPosn = [(0,i) for i in range(ncols)] + [(i,0) for i in range(nrows)] + [(i,ncols-1) for i in range(nrows)] + [(nrows-1,i) for i in range(ncols)]
wallPosn = wallPosn + [(2,5),(3,5)] + [(4,i) for i in range(3,17)] + [(i,6) for i in range(6,13)]
wallPosn = wallPosn + [(7,12),(7,13),(7,14),(15,14),(15,15),(15,16),(11,16),(11,17),(12,17),(13,17),(17,1),(17,2),(10,1),(10,2),(10,3),(10,4),(5,3),(6,3),(7,3),(12,3),(12,4),(12,5),(12,7),(13,7),(14,7),(15,7)] + [(17,i) for i in range(7,13)] + [(i,9) for i in range(6,11)] + [(i,10) for i in range(10,15)] +\
            [(i,15) for i in range(6,12)] + [(i,13) for i in range(11,16)]

mazeq1 = Maze(nrows,ncols,actionList,prob,startPosn,goalPosn,oilPosn,bumpPosn,wallPosn,oilRew,bumpRew,goalRew,emptyRew,actionRew)


In [ ]:
State_Matrix = \
    np.array([[0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 0],
              [0, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 0],
              [0, 214, 215, 216, 217,   0, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 0],
              [0, 197, 198, 199, 200,   0, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 0],
              [0, 193, 194,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 195, 196, 0],
              [0, 176, 177,   0, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 0],
              [0, 162, 163,   0, 164, 165,   0, 166, 167,   0, 168, 169, 170, 171, 172,   0, 173, 174, 175, 0],
              [0, 151, 152,   0, 153, 154,   0, 155, 156,   0, 157, 158,   0,   0,   0,   0, 159, 160, 161, 0],
              [0, 136, 137, 138, 139, 140,   0, 141, 142,   0, 143, 144, 145, 146, 147,   0, 148, 149, 150, 0],
              [0, 121, 122, 123, 124, 125,   0, 126, 127,   0, 128, 129, 130, 131, 132,   0, 133, 134, 135, 0],
              [0,   0,   0,   0,   0, 111,   0, 112, 113,   0,   0, 114, 115, 116, 117,   0, 118, 119, 120, 0],
              [0,  99, 100, 101, 102, 103,   0, 104, 105, 106,   0, 107, 108,   0, 109,   0,   0,   0, 110, 0],
              [0,  89,  90,   np.NAN,   np.NAN,   np.NAN,  np.NAN,   np.NAN,  91,  92,   0,  93,  94,   0,  95,  96,  97,   0,  98, 0],
              [0,  75,  76,  77,  78,  79,  80,   0,  81,  82,   0,  83,  84,   0,  85,  86,  87,   0,  88, 0],
              [0,  60,  61,  62,  63,  64,  65,   0,  66,  67,   0,  68,  69,   0,  70,  71,  72,  73,  74, 0],
              [0,  47,  48,  49,  50,  51,  52,   0,  53,  54,  55,  56,  57,   0,   0,   0,   0,  58,  59, 0],
              [0,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46, 0],
              [0,   0,   0,  19,  20,  21,  22,   0,   0,   0,   0,   0,   0,  23,  24,  25,  26,  27,  28, 0],
              [0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17,  18, 0],
              [0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 0]])
        

plt.subplots(figsize=(10,7.5))
heatmap = sns.heatmap(State_Matrix, fmt=".2f", linewidths=0.25, linecolor='black',
                      cbar= False, cmap= 'rocket_r')
heatmap.set_facecolor('black') # Color for the NaN cells in the state matrix
plt.title('Maze Problem')
plt.show()

def coloring_blocks(heatmap, oil_states, bump_states, start_state, end_state):
    # Adding red oil blocks
    for i in range(len(oil_states)):
        heatmap.add_patch(Rectangle((oil_states[i][1], oil_states[i][0]), 1, 1,
                                    fill=True, facecolor='red', edgecolor='red', lw=0.25))
    # Adding salmon bump blocks
    for i in range(len(bump_states)):
        heatmap.add_patch(Rectangle((bump_states[i][1], bump_states[i][0]), 1, 1,
                                    fill=True, facecolor='lightsalmon', edgecolor='lightsalmon', lw=0.25))
    # Adding start block (Blue)
    heatmap.add_patch(Rectangle((start_state[1], start_state[0]), 1, 1,
                                fill=True, facecolor='lightblue', edgecolor='lightblue', lw=0.25))

    # Adding end block (Green)
    heatmap.add_patch(Rectangle((end_state[1], end_state[0]), 1, 1,
                                fill=True, facecolor='lightgreen', edgecolor='lightgreen', lw=0.25))

# Example Use
plt.subplots(figsize=(10,7.5))    
heatmap = sns.heatmap(State_Matrix, fmt=".2f", linewidths=0.25, linecolor='black',
                      cbar= False, cmap= 'rocket_r')
heatmap.set_facecolor('black') # Color for the NaN cells in the state matrix
coloring_blocks(heatmap, oil_states=[(3,5),(4,6)], bump_states=[(13,15),(14,16)], \
                start_state=(2,2),end_state=(12,12))
plt.show()

# Q2


In [82]:
def XOR(a, b):
    if a != b:
        return 1
    else:
        return 0

def to_binary(n):
  bin_arr = [0,0,0,0]
  i = 0
  while (n>0):
      bin_arr[i] = n%2
      n = int(n/2)
      i = i+1
  bin_arr.reverse()
  return np.array(bin_arr)

def to_decimal(arr):
  bin_arr = np.flip(arr)
  mulArr = np.array([2**x for x in range(4)])
  return np.sum(bin_arr*mulArr)

def getnk(p):
  q = np.random.binomial(1,p,size=4)
  return q

def vNormalize(value):
  if value <= 0:
    return 0
  else:
    return 1

def epGreedy(classObj,s,p):
  ep = np.random.uniform(0.0,1.0)
  if ep < p:
    return classObj.randomAction()
  else:
    return classObj.greedy(s)

def SARSA(classObj,alpha,gamma):
  numEpCount = 0
  maxEpCount = 1000
  EpLength = 0
  maxEpLength = 1000
  avgReward = np.zeros(maxEpLength)
  while numEpCount < maxEpCount:
    s = classObj.randomState()
    a = epGreedy(classObj,s,p)
    EpLength = 0
    while EpLength < maxEpLength:
      s_ = classObj.nextState(s,a)
      r = classObj.getR(s,s_,a)
      avgReward[EpLength] += r 
      a_ = epGreedy(classObj,s_,p)
      qsa = classObj.getQ(s,a)
      qsa_ = classObj.getQ(s_,a_)
      delta = r + gamma*qsa_ - qsa
      qsa = qsa + alpha*delta
      classObj.setQ(s,a,qsa)
      #print("S:",s,"A:",a,"R:",r,"S-:",s_,"A-:",a_,"val:",val)
      s = s_
      a = a_
      EpLength += 1
    
    numEpCount += 1
  avgReward = avgReward/maxEpCount

def QLearn(classObj,alpha,gamma):
  numEpCount = 0
  maxEpCount = 1000
  EpLength = 0
  maxEpLength = 1000
  avgReward = np.zeros(maxEpLength)
  while numEpCount < maxEpCount:
    s = classObj.randomState()
    EpLength = 0
    while EpLength < maxEpLength:
      a = epGreedy(classObj,s,p)
      s_ = classObj.nextState(s,a)
      r = classObj.getR(s,s_,a)
      avgReward[EpLength] += r 
      qsa = classObj.getQ(s,a)
      a_ = classObj.greedy(s_)
      qsa_ = classObj.getQ(s_,a_)
      delta = r + gamma*qsa_ - qsa
      qsa = qsa + alpha*delta
      classObj.setQ(s,a,qsa)
      #print("S:",s,"A:",a,"R:",r,"S-:",s_,"A-:",a_,"val:",val)
      s = s_
      EpLength += 1
    
    numEpCount += 1
  avgReward = avgReward/maxEpCount


def SARSA_l(classObj,alpha,gamma,lamb):
  numEpCount = 0
  maxEpCount = 1000
  EpLength = 0
  maxEpLength = 1000
  avgReward = np.zeros(maxEpLength)
  while numEpCount < maxEpCount:
    s = classObj.randomState()
    a = classObj.randomAction()
    classObj.resetE()
    EpLength = 0
    while EpLength < maxEpLength:
      s_ = classObj.nextState(s,a)
      r = classObj.getR(s,s_,a)
      a_ = epGreedy(classObj,s_,p)
      qsa = classObj.getQ(s,a)
      qsa_ = classObj.getQ(s_,a_)
      delta = r + gamma*qsa_ - qsa
      classObj.Eincrement(s,a)
      avgReward[EpLength] += r 
      for pair in classObj.stateActionIter():
        s,a = pair
        e = classObj.getE(s,a)
        qsa = qsa + alpha*delta*e
        classObj.setQ(s,a,qsa)
        classObj.changeE(s,a,gamma,lamb)
      #print("S:",s,"A:",a,"R:",r,"S-:",s_,"A-:",a_,"val:",val)
      s = s_
      a = a_
      EpLength += 1
    
    numEpCount += 1
  avgReward = avgReward/maxEpCount

def AC(classObj,alpha,gamma,beta):
  numEpCount = 0
  maxEpCount = 2
  EpLength = 0
  maxEpLength = 10
  while numEpCount < maxEpCount:
    s = classObj.randomState()
    while EpLength < maxEpLength:
      classObj.updateP()
      a = np.random.choice([x for x in range(len(classObj.actions))],p=classObj.getPS(s))
      s_ = classObj.nextState(s,a)
      r = classObj.getR(s,s_,a)
      vs = classObj.getValue(s)
      vs_ = classObj.getValue(s_)
      delta = r + gamma*vs_ - vs
      vs = vs + alpha*delta
      classObj.updateH(beta,delta,s,a)
      s = s_
      EpLength += 1
    
    numEpCount += 1


In [91]:
class Genes():
  def __init__(self,actionList,prob,connectivityMatrix):
    self.actions = actionList #a1,a2,a3,a4 in that order
    self.p = prob
    self.C = connectivityMatrix
    #self.Rsas = np.array([np.zeros((16,16))] * len(self.actions))
    #self.transistion = np.array([np.zeros((16,16))] * len(self.actions))
    #self.Ras = np.zeros((len(self.actions),16))
    #self.values = np.zeros((16,1))
    self.Q = np.zeros((16,len(self.actions))) #Row is states, columns is actions
    self.e = np.zeros((16,len(self.actions))) #Row is states, columns is actions
    self.policy = np.zeros((16,)) #will store action values at 1,2,3,4
    self.values = np.zeros((16,))
    self.H = np.zeros((16,len(self.actions))) #Row is states, columns is actions
    self.P = np.zeros((16,len(self.actions))) #Row is states, columns is actions 
    
  def setACPolicy(self):
    for i in range(16):
      self.policy[i] = np.argmax(self.P[i])
  
  def getValue(self,s):
    return self.values[s]
    
  def getPS(self,s):
    return self.P[s,:]

  def updateP(self):
    for i in range(16):
      for j in range(len(actionList)):
        self.P[i,j] = np.exp(self.H[i,j])/np.sum(np.exp(self.H[i,:]))

  def updateH(self,beta,delta,s,a):
    self.H[s,a] += beta*delta*(1-self.P[s,a])

  def resetE(self):
    self.e = np.zeros((16,len(self.actions)))

  def setQ(self,s,a,val):
    self.Q[s,a] = val

  def Eincrement(self,s,a):
    self.e[s,a] += 1

  def changeE(self,s,a,gamma,l):
    self.e[s,a] = self.e[s,a]*gamma*l
  
  def getE(self,s,a):
    return self.e[s,a]
  
  def setPolicy(self):
    for i in range(16):
      self.policy[i] = np.argmax(self.Q[i,:])

  def getQ(self,s,a):
    return self.Q[s,a]

  def randomAction(self):
    return np.random.randint(0,len(self.actions))

  def randomState(self):
    return np.random.randint(0,16)

  def greedy(self,s):
    slice = self.Q[s,:]
    return np.argmax(slice)

  def getR(self,stateCurrent,stateNext,action): #State number
    stateC = to_binary(stateCurrent)
    stateN = to_binary(stateNext)
    a = self.actions[action]
    reward = 5*stateN - a
    return np.sum(reward)

  def stateIter(self):
    for i in range(16):
      yield i
  
  def stateActionIter(self):
    for i in range(16):
      for j in range(len(self.actions)):
        yield (i,j)

  def nextState(self,currentState,action):
    thisState = to_binary(currentState).reshape((4,1))
    a = self.actions[action]
    val = np.matmul(self.C,thisState).reshape((4,))
    val = np.array(list(map(vNormalize,val)))
    val = np.array(list(map(XOR,val,a)))
    nk = getnk(self.p)
    val = np.array(list(map(XOR,val,nk)))
    nextS = to_decimal(val)
    return nextS 
    

p = 0.05
C = np.array([[0,0,-1,0],[1,0,-1,-1],[0,1,0,0],[-1,1,1,0]])
actionList = [np.array([0,0,0,0]),np.array([0,1,0,0]),np.array([0,0,1,0]),np.array([0,0,0,1])]
alpha = 0.2
gamma = 0.95
beta = 0.05
l = 0.95


In [85]:
for i in range(10):
    gene1 = Genes(actionList,p,C)
    SARSA(gene1,alpha,gamma)
    gene1.setPolicy()
    print(gene1.policy)


[1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 2. 1. 3. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 0. 3. 0. 2. 0. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 3. 1. 1. 1.]
[1. 2. 1. 1. 1. 1. 1. 1. 0. 1. 2. 1. 3. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 0. 2. 0. 1. 0. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 0. 2. 3. 1. 0. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 2. 1. 3. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 2. 0. 1. 1. 1.]


In [86]:
for i in range(10):
    gene2 = Genes(actionList,p,C)
    QLearn(gene2,alpha,gamma)
    gene2.setPolicy()
    print(gene2.policy)


[1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 3. 1. 1. 1.]
[2. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 2. 1. 3. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 3. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 3. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 0. 2. 2. 1. 0. 1. 1. 1.]


In [106]:
for i in range(10):
    gene3 = Genes(actionList,p,C)
    SARSA_l(gene3,alpha,gamma,l)
    gene3.setPolicy()
    print(gene3.policy)


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[3. 2. 3. 3. 2. 3. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


KeyboardInterrupt: 

In [105]:
#for i in range(10):
for i in range(10):
    gene4 = Genes(actionList,p,C)
    AC(gene4,alpha,gamma,beta)
    gene4.setACPolicy()
    print(gene4.policy)

[1. 3. 0. 0. 0. 2. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 2. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 2. 2. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[3. 0. 1. 0. 1. 2. 1. 3. 0. 1. 0. 0. 0. 0. 0. 3.]
[0. 1. 1. 0. 0. 2. 0. 0. 0. 3. 0. 0. 0. 0. 0. 0.]
[3. 0. 1. 0. 0. 1. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 2. 0. 1. 0. 2. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[2. 2. 2. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[3. 1. 0. 0. 0. 2. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0.]
[1. 2. 0. 1. 2. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [95]:
#    for k in range(len(self.actions)):
#      for i in range(16):
#        for j in range(16):
#          statei = to_binary(i).reshape(4,1)
#          statej = to_binary(j).reshape(4,1)
#          v = np.matmul(self.C,statei)
#          v = np.array(list(map(XOR,v,self.actions[k]))).reshape(4,1)
#          v = statej - v
#          value = abs(int(sum(v)))
#          self.transistion[k,i,j] = (self.p**value)*((1-self.p)**(4-value))
#          self.Rsas[k,i,j] = 5*sum(statej) - sum(self.actions[k])
#    
#    for k in range(len(self.actions)):
#      ans = np.multiply(self.transistion[k],self.Rsas[k])
#      self.Ras[k] = np.sum(ans,axis=1)
#    
#    self.Ras = np.reshape(self.Ras,(len(self.actions),16,1))
#
#  def getTransition(self):
#    return self.transistion      
#
#  def getReward(self):
#    return self.Ras
#
#  def getActions(self):
#    return self.actions
#  
#  def getValues(self):
#    return self.values
